In [1]:
#File: dataPreprocessing.ipynb
#Purpose: data preprocessing for steve mapping dataset
#Author: Quan Gan, Yuxuan Zhou
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('steve_mapping_1000.csv', skiprows = 1)

In [3]:
info = df[['collectionCode', 'habitat', 'higherGeography', 'locality','higherClassification', 'Steve label']]
count = 0

for index,row in info.iteritems():

    #recrods' labels clean
    info = info.replace(np.nan, "", regex=True)
    info = info[info['Steve label'] != ""]
    info = info.replace('\n', " ", regex=True)
    info = info.replace('Lake,', "Lake", regex=True)
      
    #add prefix "__label__" to labels
    labels = info['Steve label'].str.split(r', ', expand=True)
    labels = labels.rename(columns= {0: "label1", 1: "label2"})
    labels['label1'] = "__label__" + labels['label1']
    labels = labels.fillna("")
    labels.loc[labels['label2'] != "", 'label2'] = "__label__"+labels.loc[labels['label2'] != ""]['label2'] 
    
    if count <= 4:
        info = info.drop(columns = [index])
    
    result = labels.replace(" ", "_", regex=True)
        
    #records' text clean
    info['text'] = np.array([ '' for i in range(995)])
    
    for new_index, new_row in info.iteritems():
        if new_index != str('text') and new_index != str('Steve label'):
            info['text'] += info[new_index] + " " 
    
    info['text'] = info['text'].str.strip()
    info['text'] = info['text'].str.replace('[^\w\s]','',regex = True)
    info['text'] = info['text'].str.lower()
     
    #sample records to trainset and testset
    result['label'] = result['label1'] + " " + result['label2']
    result['text'] = info['text']
    ds = result['label'] + " " + result['text']
    
    name = ['collectionCode', 'habitat', 'higherGeography', 'locality','higherClassification', 'full']
    ds.to_csv(r'cleanedData' + "_" + name[count] + "_remove" + '.txt', header=False, index=False)

    f = open('title.txt', 'a')  
    f.write('cleanedData' + "_" + name[count] + "_remove" + '.txt\n')
    info = df[['collectionCode', 'habitat', 'higherGeography', 'locality','higherClassification', 'Steve label']]
    count += 1
f.close()

In [4]:
# avoid the same file name appearing multiple times
p = open("title.txt", "r")
line = list(set(p.readlines()))

r = open('new_title.txt', 'w')
for i in line:
    r.write(i)

r.close()
f.close()